# **NYC Taxi Data Analysis - Yellow Taxis**

Exploratory analysis on NYC Yellow Taxis looking into customer segmentation and factors that may impact tipping behaviors across the 5 boroughs

--- 

### **Imports & Setup**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
sys.path.append('../scripts') # imports custom scripts from the scripts directory
from data_utils import rearr_datetime_cols, remove_outliers, same_zone_perc, identify_cancelled_rides, remove_cancelled_fare_pairs, categorize_zones

# setting display options for pandas
pd.set_option('display.max_columns', None) # displays all columns in the dataframe
pd.set_option('display.max_rows', 100) # sets the max number of rows to 100

### **Loading Data**

In [2]:
# look up tables
zone = pd.read_csv('../data/taxi_zone_lookup.csv') 
vendor = pd.read_csv('../data/taxi_vendor_lookup.csv')
payment = pd.read_csv('../data/payment_lookup.csv')
ratecode = pd.read_csv('../data/ratecode_lookup.csv')

# yellow taxi data
df = pd.read_parquet('../data/yellow_tripdata_2025-05.parquet')

In [3]:
df.columns  # displays all columns in the dataframe

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'Airport_fee',
       'cbd_congestion_fee'],
      dtype='object')

In [4]:
df.describe()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee
count,4.591845e+06,4591845,4591845,3.395669e+06,4.591845e+06,3.395669e+06,4.591845e+06,4.591845e+06,4.591845e+06,4.591845e+06,4.591845e+06,4.591845e+06,4.591845e+06,4.591845e+06,4.591845e+06,4.591845e+06,3.395669e+06,3.395669e+06,4.591845e+06
mean,1.875653e+00,2025-05-16 07:15:25.300312,2025-05-16 07:33:20.003034,1.295270e+00,7.653422e+00,2.432585e+00,1.611871e+02,1.611876e+02,9.123720e-01,1.835794e+01,1.164838e+00,4.774078e-01,2.858122e+00,5.114760e-01,9.563765e-01,2.688033e+01,2.197677e+00,1.531993e-01,5.278734e-01
min,1.000000e+00,2009-01-01 00:20:39,2009-01-01 00:20:49,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,-9.980000e+02,-1.739000e+01,-5.000000e-01,-9.044000e+01,-1.481700e+02,-1.000000e+00,-1.147170e+03,-2.500000e+00,-1.750000e+00,-7.500000e-01
25%,2.000000e+00,2025-05-08 18:23:12,2025-05-08 18:42:50,1.000000e+00,1.060000e+00,1.000000e+00,1.140000e+02,1.070000e+02,0.000000e+00,8.600000e+00,0.000000e+00,5.000000e-01,0.000000e+00,0.000000e+00,1.000000e+00,1.554000e+01,2.500000e+00,0.000000e+00,0.000000e+00
50%,2.000000e+00,2025-05-16 00:49:45,2025-05-16 01:05:55,1.000000e+00,1.890000e+00,1.000000e+00,1.610000e+02,1.620000e+02,1.000000e+00,1.415000e+01,0.000000e+00,5.000000e-01,2.000000e+00,0.000000e+00,1.000000e+00,2.142000e+01,2.500000e+00,0.000000e+00,7.500000e-01
75%,2.000000e+00,2025-05-23 13:28:04,2025-05-23 13:47:48,1.000000e+00,3.770000e+00,1.000000e+00,2.330000e+02,2.330000e+02,1.000000e+00,2.300000e+01,2.500000e+00,5.000000e-01,4.000000e+00,0.000000e+00,1.000000e+00,3.098000e+01,2.500000e+00,0.000000e+00,7.500000e-01
max,7.000000e+00,2025-06-01 00:04:31,2025-06-04 11:17:10,9.000000e+00,2.631040e+05,9.900000e+01,2.650000e+02,2.650000e+02,4.000000e+00,1.583600e+03,1.336000e+02,2.214000e+01,4.432100e+02,1.481700e+02,1.000000e+00,1.614290e+03,2.500000e+00,6.750000e+00,1.250000e+00
std,7.239395e-01,NaN,NaN,7.252927e-01,6.533947e+02,1.133495e+01,6.655046e+01,7.058759e+01,7.590985e-01,1.982062e+01,1.845559e+00,1.380384e-01,4.043892e+00,2.144886e+00,2.750487e-01,2.419524e+01,9.446371e-01,5.431268e-01,3.607600e-01


In [5]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee
0,1,2025-05-01 00:07:06,2025-05-01 00:24:15,1.0,3.70,1.0,N,140,202,1,18.4,4.25,0.5,4.85,0.00,1.0,29.00,2.5,0.00,0.75
1,2,2025-05-01 00:07:44,2025-05-01 00:14:27,1.0,1.03,1.0,N,234,161,1,8.6,1.00,0.5,4.30,0.00,1.0,18.65,2.5,0.00,0.75
2,2,2025-05-01 00:15:56,2025-05-01 00:23:53,1.0,1.57,1.0,N,161,234,2,10.0,1.00,0.5,0.00,0.00,1.0,15.75,2.5,0.00,0.75
3,2,2025-05-01 00:00:09,2025-05-01 00:25:29,1.0,9.48,1.0,N,138,90,1,40.8,6.00,0.5,11.70,6.94,1.0,71.94,2.5,1.75,0.75
4,2,2025-05-01 00:45:07,2025-05-01 00:52:45,1.0,1.80,1.0,N,90,231,1,10.0,1.00,0.5,1.50,0.00,1.0,17.25,2.5,0.00,0.75


---
### **Data Assumptions**
* Each row represents a single trip record
* No explicit primary key column
* Each row's uniqueness could be composed of a combination of columns (pickup, dropoff, vendor) 
  
*Note: for this exploratory analysis, a primary key is not critical, as much of the analysis is focused on aggregation patterns rather than uniquely identifying rows* 
> If necessary, could create a 'ride_id' based off a combination of columns as a primary key

---
### **Data Cleaning**

In [6]:
# shows the number of missing values per column
df.isna().sum() 


VendorID                       0
tpep_pickup_datetime           0
tpep_dropoff_datetime          0
passenger_count          1196176
trip_distance                  0
RatecodeID               1196176
store_and_fwd_flag       1196176
PULocationID                   0
DOLocationID                   0
payment_type                   0
fare_amount                    0
extra                          0
mta_tax                        0
tip_amount                     0
tolls_amount                   0
improvement_surcharge          0
total_amount                   0
congestion_surcharge     1196176
Airport_fee              1196176
cbd_congestion_fee             0
dtype: int64

In [7]:
# checking to see the unique values in the passenger_count column 
df['passenger_count'].unique() 

array([ 1.,  0.,  2.,  3.,  4.,  5.,  6.,  9.,  8., nan])

In [8]:
df[df['passenger_count'] == 0] # it's odd that there are 0's for passenger_count, doesn't make sense, investigate further
df['passenger_count'] = df['passenger_count'].replace({0: pd.NA, np.nan: pd.NA}) # doesn't seem like there was an actual reason for it, will replace 0s with nulls and replace np.nan with NA to standardize
df.drop(columns=['store_and_fwd_flag'], inplace=True) # dropping store_and_fwd_flag because it's not relevant for this specific analysis
df.isna().sum() # checking to see the number of missing values per column again after the changes

VendorID                       0
tpep_pickup_datetime           0
tpep_dropoff_datetime          0
passenger_count          1220143
trip_distance                  0
RatecodeID               1196176
PULocationID                   0
DOLocationID                   0
payment_type                   0
fare_amount                    0
extra                          0
mta_tax                        0
tip_amount                     0
tolls_amount                   0
improvement_surcharge          0
total_amount                   0
congestion_surcharge     1196176
Airport_fee              1196176
cbd_congestion_fee             0
dtype: int64

In [9]:
df['RatecodeID'].unique() 
df['RatecodeID'] = df['RatecodeID'].replace(np.nan, 99) # replace nulls with 99, as listed in the source dict
df.duplicated().sum() # checking for duplicates

0

In [ ]:
# checking lookup tables individually
zone.head(), payment.head(), ratecode.head(), vendor.head()

In [11]:
vendor = vendor.loc[:, ~vendor.columns.str.contains('^Unnamed')] # vendor has 2 unnamed columns, let's filter those out, ~ removes columns that start with 'Unnamed'

---
### **Exploratory**: The 5 boroughs of NYC

__A few notes:__

1. Borough DataFrames:
    > DataFrames named after a borough (e.g., `manhattan`, `queens`) contain only rides where **BOTH** pickup & dropoff are within that borough.
2. '_overall' DataFrames:
    > DataFrames with '_overall' (e.g., `manhattan_overall`) include any ride where **EITHER** the pickup or dropoff is in that borough.
3. '_cleaned' DataFrames:
    > DataFrames with '_cleaned' (e.g., `queens_cleaned`) have extreme outliers in trip duration **AND** distance removed. These are used for analyses that are sensitive to outliers (like distribution or clustering analysis). These are dfs are bound to have changes in other segments of this project.
4. Why Use Different DataFrames:
    > Different questions require different scopes, some analyses need clean data to avoid skewed results


In [12]:
# merging lookup tables with the taxi data (left joins because we want to retain all records from the taxi data)
merged = df.merge(zone, how='left', left_on='PULocationID', right_on='LocationID').rename(columns={'Borough': 'PU_Borough', 'Zone': 'PU_Zone', 'service_zone': 'PU_service_zone'}) # saving location information based off PULocationID 
merged = merged.merge(zone, how='left', left_on='DOLocationID', right_on='LocationID').rename(columns={'Borough': 'DO_Borough', 'Zone': 'DO_Zone', 'service_zone': 'DO_service_zone'}) # saving location information based off DOLocationID

merged = merged.drop(columns=['LocationID_x', 'LocationID_y']) # dropping LocationID columns, no longer needed
merged = merged.merge(vendor, how='left', on='VendorID').merge(payment, how='left', on='payment_type').merge(ratecode, how='left', on='RatecodeID') # merging the rest of the lookups

# create a column with the difference between pickup and dropoff datetime as 'duration_mins'
merged['duration_mins'] = (merged['tpep_dropoff_datetime'] - merged['tpep_pickup_datetime']).dt.total_seconds() / 60 # duration in minutes
merged.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee,PU_Borough,PU_Zone,PU_service_zone,DO_Borough,DO_Zone,DO_service_zone,vendor_name,payment_method,Rate,duration_mins
0,1,2025-05-01 00:07:06,2025-05-01 00:24:15,1.0,3.70,1.0,140,202,1,18.4,4.25,0.5,4.85,0.00,1.0,29.00,2.5,0.00,0.75,Manhattan,Lenox Hill East,Yellow Zone,Manhattan,Roosevelt Island,Boro Zone,"Creative Mobile Technologies, LLC",Credit card,Standard,17.150000
1,2,2025-05-01 00:07:44,2025-05-01 00:14:27,1.0,1.03,1.0,234,161,1,8.6,1.00,0.5,4.30,0.00,1.0,18.65,2.5,0.00,0.75,Manhattan,Union Sq,Yellow Zone,Manhattan,Midtown Center,Yellow Zone,"Curb Mobility, LLC",Credit card,Standard,6.716667
2,2,2025-05-01 00:15:56,2025-05-01 00:23:53,1.0,1.57,1.0,161,234,2,10.0,1.00,0.5,0.00,0.00,1.0,15.75,2.5,0.00,0.75,Manhattan,Midtown Center,Yellow Zone,Manhattan,Union Sq,Yellow Zone,"Curb Mobility, LLC",Cash,Standard,7.950000
3,2,2025-05-01 00:00:09,2025-05-01 00:25:29,1.0,9.48,1.0,138,90,1,40.8,6.00,0.5,11.70,6.94,1.0,71.94,2.5,1.75,0.75,Queens,LaGuardia Airport,Airports,Manhattan,Flatiron,Yellow Zone,"Curb Mobility, LLC",Credit card,Standard,25.333333
4,2,2025-05-01 00:45:07,2025-05-01 00:52:45,1.0,1.80,1.0,90,231,1,10.0,1.00,0.5,1.50,0.00,1.0,17.25,2.5,0.00,0.75,Manhattan,Flatiron,Yellow Zone,Manhattan,TriBeCa/Civic Center,Yellow Zone,"Curb Mobility, LLC",Credit card,Standard,7.633333


In [ ]:
merged.isna().sum() #after merging, there are some nulls in the lookup columns, which is expected since not all records will have a corresponding entry in the lookup tables

In [14]:
merged['PU_Zone'].unique(), merged['DO_Zone'].unique(), merged['PU_Borough'].unique(), merged['DO_Borough'].unique() # checking unique values in the merged columns
merged['PU_Borough'] = merged['PU_Borough'].replace(np.nan, 'Unknown')
merged['DO_Borough'] = merged['DO_Borough'].replace(np.nan, 'Unknown') # there are 'Unknown' and 'NaN' values in PU_Borough & DO_Borough, replace na with 'Unknown' for consistency

#### **Understanding the Data (Geographic & Trip Characteristics)**
>How do often people request rides, where do they request the most?
* Focusing on location to perform aggregation

In [15]:
ride_combination = (merged.groupby(['PU_Borough', 'DO_Borough']).size().reset_index(name='count').sort_values(by='count',ascending=False)) # count occurrences of rides requested from PU_Borough and DO_Borough & sorts 
ride_combination.head() # by a good chunk, the most frequent combination is from Manhattan to Manhattan

,PU_Borough,DO_Borough,count
23,Manhattan,Manhattan,3657462
30,Queens,Manhattan,225823
31,Queens,Queens,136701
24,Manhattan,Queens,114579
8,Brooklyn,Brooklyn,104316


*As shown above:*
> The results shows Manhattan having the most ride frequency, let's investigate Manhattan a bit further:
  * Could there be a trend since most rides are within Manhattan? Consider:
    * Yellow taxis could be commonly used for short trips strictly within Manhattan 
    * Take a closer look at the datetime & trip distance to confirm the durations of these rides with its corresponding distance


In [16]:
# Manhattan ONLY (PU_Borough & DO_Borough both equal 'Manhattan')
manhattan = merged[(merged['PU_Borough'] == 'Manhattan') & (merged['DO_Borough'] == 'Manhattan')].rename(columns={'tpep_pickup_datetime': 'PU_datetime', 'tpep_dropoff_datetime': 'DO_datetime'})

# reindexing to have the columns in a more readable order
manhattan_datetime_cols = ['PU_datetime', 'DO_datetime', 'duration_mins', 'trip_distance', 'passenger_count', 'PU_Zone', 'PU_service_zone', 'DO_Zone', 'DO_service_zone']
manhattan = manhattan[manhattan_datetime_cols + [c for c in manhattan.columns if c not in manhattan_datetime_cols]] 
manhattan.head() # checking after reindexing

,PU_datetime,DO_datetime,duration_mins,trip_distance,passenger_count,PU_Zone,PU_service_zone,DO_Zone,DO_service_zone,VendorID,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee,PU_Borough,DO_Borough,vendor_name,payment_method,Rate
0,2025-05-01 00:07:06,2025-05-01 00:24:15,17.150000,3.70,1.0,Lenox Hill East,Yellow Zone,Roosevelt Island,Boro Zone,1,1.0,140,202,1,18.4,4.25,0.5,4.85,0.0,1.0,29.00,2.5,0.0,0.75,Manhattan,Manhattan,"Creative Mobile Technologies, LLC",Credit card,Standard
1,2025-05-01 00:07:44,2025-05-01 00:14:27,6.716667,1.03,1.0,Union Sq,Yellow Zone,Midtown Center,Yellow Zone,2,1.0,234,161,1,8.6,1.00,0.5,4.30,0.0,1.0,18.65,2.5,0.0,0.75,Manhattan,Manhattan,"Curb Mobility, LLC",Credit card,Standard
2,2025-05-01 00:15:56,2025-05-01 00:23:53,7.950000,1.57,1.0,Midtown Center,Yellow Zone,Union Sq,Yellow Zone,2,1.0,161,234,2,10.0,1.00,0.5,0.00,0.0,1.0,15.75,2.5,0.0,0.75,Manhattan,Manhattan,"Curb Mobility, LLC",Cash,Standard
4,2025-05-01 00:45:07,2025-05-01 00:52:45,7.633333,1.80,1.0,Flatiron,Yellow Zone,TriBeCa/Civic Center,Yellow Zone,2,1.0,90,231,1,10.0,1.00,0.5,1.50,0.0,1.0,17.25,2.5,0.0,0.75,Manhattan,Manhattan,"Curb Mobility, LLC",Credit card,Standard
6,2025-05-01 00:18:14,2025-05-01 00:27:38,9.400000,1.50,<NA>,Lenox Hill East,Yellow Zone,Yorkville West,Yellow Zone,1,1.0,140,263,1,11.4,3.50,0.5,4.05,0.0,1.0,20.45,2.5,0.0,0.00,Manhattan,Manhattan,"Creative Mobile Technologies, LLC",Credit card,Standard


> Finding the % of manhattan to manhattan rides that have short durations or trip distances
* take a look at the distribution of trip durations & distances 
* filter out negative value into 'manhattan_cleaned', likely input error 

In [17]:
manhattan_cleaned = manhattan[manhattan['duration_mins'] >= 0]
manhattan_cleaned[['duration_mins', 'trip_distance']].describe(), merged[['trip_distance', 'duration_mins']].describe()

(       duration_mins  trip_distance
 count   3.657438e+06   3.657438e+06
 mean    1.406173e+01   4.643766e+00
 std     2.414163e+01   5.236403e+02
 min     0.000000e+00   0.000000e+00
 25%     7.550000e+00   9.500000e-01
 50%     1.208333e+01   1.580000e+00
 75%     1.816667e+01   2.560000e+00
 max     7.664717e+03   2.631040e+05,
        trip_distance  duration_mins
 count   4.591845e+06   4.591845e+06
 mean    7.653422e+00   1.791171e+01
 std     6.533947e+02   2.799913e+01
 min     0.000000e+00  -6.810833e+02
 25%     1.060000e+00   8.366667e+00
 50%     1.890000e+00   1.391667e+01
 75%     3.770000e+00   2.216667e+01
 max     2.631040e+05   7.664717e+03)

In [18]:
merged_cleaned = merged[merged['duration_mins'] >= 0] # let's also clean the merged df to remove negative values

# we can use values <= the 25th percentile of the overall df to define short rides by durations OR distances
duration_25_all = merged_cleaned['duration_mins'].quantile(0.25)
distance_25_all = merged_cleaned['trip_distance'].quantile(0.25)

# defining short rides in Manhattan (all rides with short durations OR distances)
m_short_rides = manhattan_cleaned[(manhattan_cleaned['duration_mins'] <= duration_25_all) | (manhattan_cleaned['trip_distance'] <= distance_25_all)] 
m_short_rides_percentage = (m_short_rides.shape[0] / manhattan_cleaned.shape[0]) * 100
print(f"% of short rides in Manhattan (by duration or distance): {m_short_rides_percentage:.2f}%")

% of short rides in Manhattan (by duration or distance): 38.83%


>It doesn't necessarily seem like yellow taxis are used mainly for short trips within Manhattan
* Look from a broader perspective to display zones pairs where rides are frequently being called from and to

In [19]:
# rides involving Manhattan (PU_Borough or DO_Borough equal 'Manhattan')
manhattan_overall = merged[(merged['PU_Borough'] == 'Manhattan') | (merged['DO_Borough'] == 'Manhattan')].rename(columns={'tpep_pickup_datetime': 'PU_datetime', 'tpep_dropoff_datetime': 'DO_datetime'})
manhattan_overall.groupby(['PU_Zone', 'DO_Zone']).size().reset_index(name='count').sort_values(by='count', ascending=False).head()  

,PU_Zone,DO_Zone,count
18782,Upper East Side South,Upper East Side North,29651
18547,Upper East Side North,Upper East Side South,25399
18783,Upper East Side South,Upper East Side South,20890
18546,Upper East Side North,Upper East Side North,19259
12306,Midtown Center,Upper East Side South,13813


>Could be useful to look into Same-Zone Rides in Manhattan vs. Manhattan-Involved Rides

In [20]:
# checking the amount of rides within the same zone for ride within Manhattan (PU_Borough & DO_Borough equals Manhattan, AND PU_Zone == DO_Zone)
m_same_zone_percentage = (manhattan[manhattan['PU_Zone'] == manhattan['DO_Zone']].shape[0]/manhattan.shape[0])*100  # percentage of rides within the same zone in Manhattan
print(f"% of same zone rides within Manhattan: {m_same_zone_percentage:.2f}%")

% of same zone rides within Manhattan: 4.70%


>It's a low percentage:
  * Maybe there are less local taxi rides because the borough is more walkable?
    * The numbers are on the lower end for duration & distance
    * Could indicate most taxi rides are cross-neighborhood or intra-borough
    * Will compare with other boroughs to see if Manhattan is unique in this behavior

In [21]:
# checking the amount of rides within the same zone for all Manhattan-involved rides (PU_Borough | DO_Borough equals 'Manhattan' AND, PU_Zone == DO_Zone)
m_same_zone_percentage = (manhattan[manhattan['PU_Zone'] == manhattan['DO_Zone']].shape[0]/manhattan_overall.shape[0])*100  # using manhattan_overall instead
print(f"% of same zone rides involving Manhattan: {m_same_zone_percentage:.2f}%")

% of same zone rides involving Manhattan: 4.09%


> The percentage is low for both, and even lower when considering all Manhattan-involved rides
* This demonstrates that most yellow taxi rides in Manhattan are not hyper-local, but instead cross zones or neighborhoods
* Could support the idea that Manhattan's walkability reduces the need for taxis for very short and local trips due to the borough being less spaced out

> *Note: I'll be saving a few functions from above in the data_utils pyscript since some will be applied to the rest of the boroughs*
* **'rearr_datetime_cols':**
  * referring to the reindexing/reordering of columns to have the dataframe in a readable manner
* **'remove_outliers':**
  * filters negative values out for 'duration_mins' & 'trip_distance'
  * then finds the 1 and 99th quantile based off those 2 columns and removes those values
* **'same_zone_perc':**
  * returns the percentage of rides within the same zone strictly and involving the borough

>Analyzing the rest of the boroughs: Queens

In [22]:
# ONLY Queens (PU_Borough and DO_Borough both equal 'Queens')
queens = merged[(merged['PU_Borough'] == 'Queens') & (merged['DO_Borough'] == 'Queens')].rename(columns={'tpep_pickup_datetime': 'PU_datetime', 'tpep_dropoff_datetime': 'DO_datetime'})
queens = rearr_datetime_cols(queens)
queens.head()

,PU_datetime,DO_datetime,duration_mins,trip_distance,passenger_count,PU_Zone,PU_service_zone,DO_Zone,DO_service_zone,VendorID,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee,PU_Borough,DO_Borough,vendor_name,payment_method,Rate
5,2025-05-01 00:09:24,2025-05-01 00:22:04,12.666667,5.11,1.0,LaGuardia Airport,Airports,Sunnyside,Boro Zone,2,1.0,138,226,1,22.6,6.00,0.5,6.02,0.0,1.0,37.87,0.0,1.75,0.0,Queens,Queens,"Curb Mobility, LLC",Credit card,Standard
54,2025-05-01 00:39:32,2025-05-01 01:13:48,34.266667,16.51,1.0,JFK Airport,Airports,JFK Airport,Airports,2,1.0,132,132,1,66.0,1.00,0.5,17.12,0.0,1.0,85.62,0.0,0.00,0.0,Queens,Queens,"Curb Mobility, LLC",Credit card,Standard
126,2025-05-01 00:11:11,2025-05-01 00:24:54,13.716667,4.53,2.0,JFK Airport,Airports,Richmond Hill,Boro Zone,2,1.0,132,197,1,20.5,1.00,0.5,6.90,0.0,1.0,31.65,0.0,1.75,0.0,Queens,Queens,"Curb Mobility, LLC",Credit card,Standard
148,2025-05-01 00:29:36,2025-05-01 01:06:47,37.183333,9.18,2.0,JFK Airport,Airports,Hollis,Boro Zone,2,1.0,132,122,2,43.6,1.00,0.5,0.00,0.0,1.0,47.85,0.0,1.75,0.0,Queens,Queens,"Curb Mobility, LLC",Cash,Standard
172,2025-05-01 00:13:08,2025-05-01 00:30:13,17.083333,7.20,1.0,LaGuardia Airport,Airports,Kew Gardens,Boro Zone,1,1.0,138,134,2,29.6,7.75,0.5,0.00,0.0,1.0,38.85,0.0,1.75,0.0,Queens,Queens,"Creative Mobile Technologies, LLC",Cash,Standard


> It seems like there's a lot of service for airports (LaGuardia & JFK) strictly within Queens
* Worth noting that both airports are located in Queens, could explain the high volume of airport type services

In [23]:
# checking the amount of rides in each service_zone
q_PU_service_zones = queens['PU_service_zone'].value_counts().rename('PU_count')  
q_DO_service_zones = queens['DO_service_zone'].value_counts().rename('DO_count')  
pd.concat([q_PU_service_zones, q_DO_service_zones], axis=1) # side-by-side counts of PU and DO service zones for Queens

,PU_count,DO_count
Boro Zone,70585,114926
Airports,66116,21775


> Looks like there's more airport services at pickup compared to dropoff

In [24]:
# rides involving Queens (PU_Borough or DO_Borough equal 'Queens')
queens_overall = merged[(merged['PU_Borough'] == 'Queens') | (merged['DO_Borough'] == 'Queens')].rename(columns={'tpep_pickup_datetime': 'PU_datetime', 'tpep_dropoff_datetime': 'DO_datetime'})

# the percentage of rides that are airport-related
q_airport_rides = queens_overall[queens_overall['PU_service_zone'].str.contains('Airport') | queens_overall['DO_service_zone'].str.contains('Airport')].shape[0]
q_airport_percentage = (q_airport_rides / queens_overall.shape[0]) * 100
print(f"% of airport-related rides in queens: {q_airport_percentage:.2f}%")

% of airport-related rides in queens: 67.23%


In [25]:
# compared to manhattan, Queens has a higher percentage of airport-related rides
m_airport_rides = manhattan_overall[manhattan_overall['PU_service_zone'].str.contains('Airport') | manhattan_overall['DO_service_zone'].str.contains('Airport')].shape[0]
m_airport_percentage = (m_airport_rides / manhattan_overall.shape[0]) * 100
print(f"% of airport-related rides in manhattan: {m_airport_percentage:.2f}%")

% of airport-related rides in manhattan: 6.20%


> In total, over half the rides involving queens are airport-related
* Could be that queens uses taxi services for lesser casual means, potentially longer trips

In [26]:
# let's filter out the extreme outliers in both columns
queens_cleaned = remove_outliers(queens)
queens_cleaned[['duration_mins', 'trip_distance']].describe()

,duration_mins,trip_distance
count,120305.000000,120305.000000
mean,18.218662,5.671123
std,11.372003,3.965334
min,0.100000,0.010000
25%,10.383333,2.430000
50%,16.233333,4.900000
75%,23.983333,8.480000
max,61.050000,16.570000


*As shown above:*
* The duration is much higher than that of Manhattan, which could be due to the fact that Queens is less densely populated and has longer distances between locations (less "walkable")
* Confirms taxi services are used longer trips, possibly for airport-related services or other longer-distance travel needs

>Analyzing the rest of the boroughs: Brooklyn

In [27]:
# ONLY Brooklyn (PU_Borough & DO_Borough both equal 'Brooklyn')
brooklyn = merged[(merged['PU_Borough'] == 'Brooklyn') & (merged['DO_Borough'] == 'Brooklyn')].rename(columns={'tpep_pickup_datetime': 'PU_datetime', 'tpep_dropoff_datetime': 'DO_datetime'})
# rides involving Brooklyn (PU_Borough or DO_Borough equal 'Brooklyn')
brooklyn_overall = merged[(merged['PU_Borough'] == 'Brooklyn') | (merged['DO_Borough'] == 'Brooklyn')].rename(columns={'tpep_pickup_datetime': 'PU_datetime', 'tpep_dropoff_datetime': 'DO_datetime'})
brooklyn = rearr_datetime_cols(brooklyn) # reindexing to have the columns in a more readable order
brooklyn.head()

,PU_datetime,DO_datetime,duration_mins,trip_distance,passenger_count,PU_Zone,PU_service_zone,DO_Zone,DO_service_zone,VendorID,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee,PU_Borough,DO_Borough,vendor_name,payment_method,Rate
266,2025-05-01 00:00:07,2025-05-01 00:22:57,22.833333,3.91,1.0,Boerum Hill,Boro Zone,Sunset Park West,Boro Zone,2,1.0,25,228,2,24.0,1.0,0.5,0.00,0.0,1.0,26.50,0.0,0.0,0.0,Brooklyn,Brooklyn,"Curb Mobility, LLC",Cash,Standard
313,2025-05-01 00:28:24,2025-05-01 00:34:01,5.616667,1.00,1.0,Williamsburg (South Side),Boro Zone,Williamsburg (South Side),Boro Zone,1,1.0,256,256,1,7.2,1.0,0.5,4.00,0.0,1.0,13.70,0.0,0.0,0.0,Brooklyn,Brooklyn,"Creative Mobile Technologies, LLC",Credit card,Standard
484,2025-05-01 00:50:08,2025-05-01 00:56:34,6.433333,1.03,1.0,Williamsburg (North Side),Boro Zone,East Williamsburg,Boro Zone,2,1.0,255,80,1,7.9,1.0,0.5,3.12,0.0,1.0,13.52,0.0,0.0,0.0,Brooklyn,Brooklyn,"Curb Mobility, LLC",Credit card,Standard
567,2025-05-01 00:12:59,2025-05-01 00:26:24,13.416667,2.80,1.0,Prospect Park,Boro Zone,Flatbush/Ditmas Park,Boro Zone,2,1.0,190,89,1,15.6,1.0,0.5,3.62,0.0,1.0,21.72,0.0,0.0,0.0,Brooklyn,Brooklyn,"Curb Mobility, LLC",Credit card,Standard
760,2025-05-01 00:54:25,2025-05-01 01:10:29,16.066667,4.51,1.0,Williamsburg (North Side),Boro Zone,Carroll Gardens,Boro Zone,2,1.0,255,40,1,21.9,1.0,0.5,4.88,0.0,1.0,29.28,0.0,0.0,0.0,Brooklyn,Brooklyn,"Curb Mobility, LLC",Credit card,Standard


In [28]:
# let's filter out the extreme outliers in both columns
brooklyn_cleaned = remove_outliers(brooklyn)
brooklyn_cleaned[['duration_mins', 'trip_distance']].describe()

,duration_mins,trip_distance
count,94404.000000,94404.000000
mean,20.098065,3.424336
std,10.882119,2.247111
min,2.316667,0.010000
25%,12.150000,1.800000
50%,18.133333,3.000000
75%,25.833333,4.440000
max,65.250000,13.200000


> It seems within Brooklyn, the duration and distance is longer than Manhattan, but slightly similar to Queens
* Look into most common pickup and dropoff zones
* Short vs. long rides

In [29]:
# checking the top paired zones in Brooklyn
brooklyn.groupby(['PU_Zone','DO_Zone']).size().reset_index(name='counts').sort_values(by='counts', ascending=False).head()  

,PU_Zone,DO_Zone,counts
1570,East New York,East New York,655
1098,Crown Heights North,Crown Heights North,434
2652,Park Slope,Park Slope,403
747,Canarsie,Canarsie,400
2120,Greenpoint,Greenpoint,349


In [30]:
same_zone_perc(brooklyn, brooklyn_overall, 'Brooklyn') # numbers seem small, checking the percentage of rides within the same zone for rides strictly within Brooklyn

% of same zone rides within Brooklyn: 7.45%
% of same zone rides involving Brooklyn: 2.25%


> Could be that Brooklyn is a borough that's considered "walkable" due to the low % 
* But if that's the case, why are the numbers so different than Manhattan?
  * Most of the rides may be for intra-borough trips, perhaps Brooklyn's neighborhoods are spaced further apart due to the larger numbers
    * Compare short rides vs. long rides in Brooklyn
    * We can use the same 25th percentile values for duration and distance as we did for Manhattan


In [31]:
# defining short rides in Brooklyn
b_short_rides = brooklyn_cleaned[(brooklyn_cleaned['duration_mins'] <= duration_25_all) | (brooklyn_cleaned['trip_distance'] <= distance_25_all)] # all rides with short durations or distances
b_short_rides_percentage = (b_short_rides.shape[0] / brooklyn_cleaned.shape[0]) * 100
print(f"% of short rides in Brooklyn (by duration or distance): {b_short_rides_percentage:.2f}%")

# defining long rides in Brooklyn
duration_75_all = merged['duration_mins'].quantile(0.75)
distance_75_all = merged['trip_distance'].quantile(0.75)

# using & instead of | to define long rides to take into account impact of delays and traffic
b_long_rides = brooklyn_cleaned[(brooklyn_cleaned['duration_mins'] >= duration_75_all) & (brooklyn_cleaned['trip_distance'] >= distance_75_all)] # all rides with long durations and distances
b_long_rides_percentage = (b_long_rides.shape[0] / brooklyn_cleaned.shape[0]) * 100
print(f"% of long rides in Brooklyn (by duration and distance): {b_long_rides_percentage:.2f}%")

% of short rides in Brooklyn (by duration or distance): 14.74%
% of long rides in Brooklyn (by duration and distance): 25.79%


In [32]:
# compared to Manhattan, Brooklyn has a higher percentage of longer rides
m_long_rides = manhattan_cleaned[(manhattan_cleaned['duration_mins'] >= duration_75_all) & (manhattan_cleaned['trip_distance'] >= distance_75_all)] # all rides with long durations and distances
m_long_rides_percentage = (m_long_rides.shape[0] / manhattan_cleaned.shape[0]) * 100
print(f"% of long rides in Manhattan (by duration and distance): {m_long_rides_percentage:.2f}%")

% of long rides in Manhattan (by duration and distance): 6.67%


> Brooklyn having a bigger % of longer rides show that, on average, it's likely to be more spaced out and less "walkable" compared to Manhattan. The smaller % of short rides also reinforces that taxi trips are likely to be longer than Manhattan.

In [33]:
# checking for non airport-related rides in Brooklyn
b_is_airport = (brooklyn_overall['PU_service_zone'].str.contains('Airport') | brooklyn_overall['DO_service_zone'].str.contains('Airport'))
non_airport_rides = brooklyn_overall[~b_is_airport]  # number of rides that are not airport-related
print(f"% of non-airport rides in Brooklyn: {non_airport_rides.shape[0]/brooklyn_overall.shape[0] * 100:.2f}%")

% of non-airport rides in Brooklyn: 85.81%


>Analyzing the rest of the boroughs: Bronx

In [34]:
# ONLY Bronx (PU_Borough & DO_Borough both equal 'Bronx')
bronx = merged[(merged['PU_Borough'] == 'Bronx') & (merged['DO_Borough'] == 'Bronx')].rename(columns={'tpep_pickup_datetime': 'PU_datetime', 'tpep_dropoff_datetime': 'DO_datetime'})
bronx = rearr_datetime_cols(bronx) # reindexing to have the columns in a more readable order
bronx.head()

,PU_datetime,DO_datetime,duration_mins,trip_distance,passenger_count,PU_Zone,PU_service_zone,DO_Zone,DO_service_zone,VendorID,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee,PU_Borough,DO_Borough,vendor_name,payment_method,Rate
3809,2025-05-01 02:53:48,2025-05-01 02:53:58,0.166667,0.02,1.0,West Concourse,Boro Zone,West Concourse,Boro Zone,2,5.0,247,247,4,-42.2,-1.0,-0.5,0.0,0.0,-1.0,-47.95,-2.5,0.0,-0.75,Bronx,Bronx,"Curb Mobility, LLC",Dispute,Negotiated fare
3810,2025-05-01 02:53:48,2025-05-01 02:53:58,0.166667,0.02,1.0,West Concourse,Boro Zone,West Concourse,Boro Zone,2,5.0,247,247,4,42.2,1.0,0.5,0.0,0.0,1.0,47.95,2.5,0.0,0.75,Bronx,Bronx,"Curb Mobility, LLC",Dispute,Negotiated fare
4388,2025-05-01 04:58:21,2025-05-01 05:08:25,10.066667,4.28,1.0,Hunts Point,Boro Zone,Mount Hope,Boro Zone,2,1.0,126,169,1,18.4,1.0,0.5,0.0,0.0,1.0,20.90,0.0,0.0,0.00,Bronx,Bronx,"Curb Mobility, LLC",Credit card,Standard
6589,2025-05-01 06:56:33,2025-05-01 08:05:18,68.750000,9.20,1.0,Morrisania/Melrose,Boro Zone,Van Nest/Morris Park,Boro Zone,1,99.0,167,242,1,27.5,0.0,0.5,0.0,0.0,0.0,28.00,0.0,0.0,0.00,Bronx,Bronx,"Creative Mobile Technologies, LLC",Credit card,Null/Unknown
6856,2025-05-01 06:49:55,2025-05-01 07:44:07,54.200000,6.30,1.0,Morrisania/Melrose,Boro Zone,Norwood,Boro Zone,1,99.0,167,174,1,28.5,0.0,0.5,0.0,0.0,0.0,29.00,0.0,0.0,0.00,Bronx,Bronx,"Creative Mobile Technologies, LLC",Credit card,Null/Unknown


In [35]:
# let's filter out the extreme outliers in both columns
bronx_cleaned = remove_outliers(bronx)
bronx_cleaned[['duration_mins', 'trip_distance']].describe()

,duration_mins,trip_distance
count,17419.000000,17419.000000
mean,16.286914,3.738788
std,7.685055,2.255315
min,2.233333,0.020000
25%,10.916667,1.900000
50%,15.116667,3.360000
75%,20.283333,5.190000
max,47.166667,10.710000


In [36]:
# top 10 most frequent ride combinations strictly within Bronx
bronx_cleaned.groupby(['PU_Zone', 'DO_Zone']).size().reset_index(name='count').sort_values(by='count', ascending=False).head(10)  

,PU_Zone,DO_Zone,count
800,Mott Haven/Port Morris,West Concourse,60
782,Mott Haven/Port Morris,Melrose South,58
704,Melrose South,Mott Haven/Port Morris,58
803,Mott Haven/Port Morris,Williamsbridge/Olinville,57
1167,Soundview/Castle Hill,Soundview/Castle Hill,56
784,Mott Haven/Port Morris,Mott Haven/Port Morris,54
775,Mott Haven/Port Morris,East Tremont,54
774,Mott Haven/Port Morris,East Concourse/Concourse Village,53
785,Mott Haven/Port Morris,Mount Hope,51
742,Morrisania/Melrose,Melrose South,51


In [37]:
# rides involving Bronx (PU_Borough or DO_Borough equal 'Bronx')
bronx_overall = merged[(merged['PU_Borough'] == 'Bronx') | (merged['DO_Borough'] == 'Bronx')].rename(columns={'tpep_pickup_datetime': 'PU_datetime', 'tpep_dropoff_datetime': 'DO_datetime'})
bronx_overall.groupby(['PU_Zone', 'DO_Zone']).size().reset_index(name='count').sort_values(by='count', ascending=False).head(10)  

,PU_Zone,DO_Zone,count
4234,JFK Airport,Spuyten Duyvil/Kingsbridge,227
4230,JFK Airport,Riverdale/North Riverdale/Fieldston,207
4223,JFK Airport,Mott Haven/Port Morris,204
4587,LaGuardia Airport,Spuyten Duyvil/Kingsbridge,178
4576,LaGuardia Airport,Mott Haven/Port Morris,167
4213,JFK Airport,East Concourse/Concourse Village,158
5890,Mott Haven/Port Morris,Mott Haven/Port Morris,153
4231,JFK Airport,Schuylerville/Edgewater Park,147
5791,Mott Haven/Port Morris,Central Harlem North,146
4583,LaGuardia Airport,Riverdale/North Riverdale/Fieldston,139


In [38]:
same_zone_perc(bronx, bronx_overall, 'Bronx') # checking the percentage of rides within the same zone for rides strictly within Bronx

% of same zone rides within Bronx: 6.60%
% of same zone rides involving Bronx: 1.74%


In [39]:
bronx_overall['PU_service_zone'].value_counts() | bronx_overall['DO_service_zone'].value_counts()  # not much airport-related rides in Bronx

PU_service_zone
Boro Zone      117587
Yellow Zone     15615
Airports         5755
EWR                 5
Name: count, dtype: int64

>Analyzing the rest of the boroughs: Staten Island

In [40]:
# ONLY Staten Island (PU_Borough & DO_Borough both equal 'Staten Island')
staten_island = merged[(merged['PU_Borough'] == 'Staten Island') & (merged['DO_Borough'] == 'Staten Island')].rename(columns={'tpep_pickup_datetime': 'PU_datetime', 'tpep_dropoff_datetime': 'DO_datetime'})
staten_island = rearr_datetime_cols(staten_island) # reindexing to have the columns in a more readable order
staten_island.head()

,PU_datetime,DO_datetime,duration_mins,trip_distance,passenger_count,PU_Zone,PU_service_zone,DO_Zone,DO_service_zone,VendorID,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee,PU_Borough,DO_Borough,vendor_name,payment_method,Rate
161085,2025-05-02 12:32:44,2025-05-02 12:45:14,12.500000,4.51,1.0,Bloomfield/Emerson Hill,Boro Zone,Mariners Harbor,Boro Zone,2,1.0,23,156,2,19.8,0.0,0.5,0.0,0.0,1.0,21.3,0.0,0.0,0.0,Staten Island,Staten Island,"Curb Mobility, LLC",Cash,Standard
185265,2025-05-02 16:05:37,2025-05-02 16:49:46,44.150000,9.01,1.0,Saint George/New Brighton,Boro Zone,Heartland Village/Todt Hill,Boro Zone,2,1.0,206,118,1,47.1,2.5,0.5,10.0,0.0,1.0,61.1,0.0,0.0,0.0,Staten Island,Staten Island,"Curb Mobility, LLC",Credit card,Standard
205941,2025-05-02 18:45:29,2025-05-02 18:52:10,6.683333,1.20,1.0,Stapleton,Boro Zone,Stapleton,Boro Zone,1,1.0,221,221,2,7.9,2.5,0.5,0.0,0.0,1.0,11.9,0.0,0.0,0.0,Staten Island,Staten Island,"Creative Mobile Technologies, LLC",Cash,Standard
258868,2025-05-03 05:19:45,2025-05-03 05:19:52,0.116667,0.00,1.0,South Beach/Dongan Hills,Boro Zone,South Beach/Dongan Hills,Boro Zone,2,5.0,214,214,2,80.0,0.0,0.5,0.0,0.0,1.0,81.5,0.0,0.0,0.0,Staten Island,Staten Island,"Curb Mobility, LLC",Cash,Negotiated fare
258869,2025-05-03 05:20:56,2025-05-03 05:21:00,0.066667,0.00,1.0,South Beach/Dongan Hills,Boro Zone,South Beach/Dongan Hills,Boro Zone,2,5.0,214,214,2,82.0,0.0,0.5,0.0,0.0,1.0,83.5,0.0,0.0,0.0,Staten Island,Staten Island,"Curb Mobility, LLC",Cash,Negotiated fare


In [41]:
# there's not much trip data within Staten Island
staten_island.shape[0]

139

In [42]:
# rides involving Staten Island (PU_Borough or DO_Borough equal 'Staten Island')
staten_island_overall = merged[(merged['PU_Borough'] == 'Staten Island') | (merged['DO_Borough'] == 'Staten Island')].rename(columns={'tpep_pickup_datetime': 'PU_datetime', 'tpep_dropoff_datetime': 'DO_datetime'})
staten_island_overall.shape[0]  # checking the number of rides involving Staten Island

1732

> There are more rides that are inter-borough vs. within borough

In [43]:
# let's filter out the extreme outliers in both columns
staten_island_cleaned = remove_outliers(staten_island)
staten_island_cleaned[['duration_mins', 'trip_distance']].describe()

,duration_mins,trip_distance
count,112.000000,112.000000
mean,11.154911,2.859464
std,6.803384,1.945837
min,0.966667,0.040000
25%,6.512500,1.330000
50%,9.666667,2.210000
75%,15.120833,4.105000
max,36.966667,8.110000


In [44]:
# comparing to the overall Staten Island rides
staten_island_overall_cleaned = remove_outliers(staten_island_overall)
staten_island_overall_cleaned[['duration_mins', 'trip_distance']].describe()

,duration_mins,trip_distance
count,1611.000000,1611.000000
mean,44.503311,17.945034
std,22.641577,8.282682
min,3.033333,0.760000
25%,29.516667,11.900000
50%,42.166667,18.490000
75%,57.025000,24.770000
max,126.566667,35.500000


In [45]:
# checking the distribution of dropoff boroughs for Staten Island rides
staten_island_overall['DO_Borough'].value_counts()

DO_Borough
Staten Island    1367
Brooklyn          264
Manhattan          78
Queens             19
Bronx               3
Unknown             1
Name: count, dtype: int64

> Compared to the rest of the boroughs, it has the shortest in terms of duration and distance within the borough. But when we take a look at the inter-borough trips, it increases 10x

> Additionally, much of the drop off points are to Staten Island
* Could be related to more traffic the closer to the center of the city or bridges

>#### **Key Findings:**
1. Manhattan has the most ride frequency out of all the 5 boroughs, followed by Queens, Brooklyn, Bronx, Staten Island in that order

2. Most yellow taxi rides in Manhattan are not hyper-local, but instead cross zones or neighborhoods. Instead of being commonly used for short trips within Manhattan, the borough is naturally closer together in distance compared to the rest of the boroughs 

3. Brooklyn & Queens seem to have longer trips than Manhattan, could be geographically more spaced out, whereas Manhattan seems more “walkable”. Therefore, Manhattan's “walkability” reduces the need for taxis for very short, local trips

4. There’s a high volume of airport type services within Queens due to JFK & LaGuardia being located there, taxis are most likely used for less casual means than that of Manhattan

5. As a whole, Taxi services seem to be less popular in Bronx, and especially in Staten Island 

6. Potential relationships with the mismatch in duration and distance to traffic, factors such as being closer to the city, or the major bridges 



---
**Note:** *Before going into analysis with fares & tips, it's important to note that there may be values that are negative, some might just be cancelled fares which should have a duplicate value but in a positive value*
> From here on out:
* **'_cleaned'** will be filtered out of cancelled rides AND unrelated negative fares to be used for aggregation as a whole for less skewed data and more accurate comparisons for fares, tips, and fees
* Will be **dropping 'EWR' columns** since fare & tip analysis is solely on the 5 boroughs, we will keep 'Unknown' columns since we can't rule out that the data could be potentially from the 5 boroughs or not, but it will be excluded from borough-level comparisons

---
#### **1. Location Segmentation & Tipping Behavior**  


> Specific zones by boroughs 
* Recall the borough specific dataframes from above, we'll be repurposing them
* Taking a look at an overview of the fare amounts (both overall & borough specific)


*Note: I'm using functions in the data_utils pyscript*
* **'identify_cancelled_rides'**
  * it filters for negative and positive fares individually, then merges together on a combination of columns that serves as a unique identifier for each ride
  * then it takes the fare_amount and matches it to its opposite pair (the same value but negative & positive)
* **'remove_cancelled_fare_pairs'**
  * it takes the matched pairs and removes it from the dataframe, along with any remaining negative fares not associated with a positive pair

In [ ]:
#first, filtering out the canceled/refunded rides to get accurate aggregations
merged.rename(columns={'tpep_pickup_datetime': 'PU_datetime', 'tpep_dropoff_datetime': 'DO_datetime'}, inplace=True) # renaming the columns for the functions to work properly

# droping all rows where borough is 'EWR' since this analysis is focused on the 5 boroughs
# will keep 'Unknown' columns since we can't rule out that the data could be potentially from the 5 boroughs or not, but it will be excluded from borough-level comparisons
merged = merged[(merged['PU_Borough'] != 'EWR') & (merged['DO_Borough'] != 'EWR')]

# identifying cancelled rides: these are rides with negative fare amounts that have corresponding positive fare amounts
overall_canceled_rides = identify_cancelled_rides(merged)
overall_canceled_rides.head(5)  

,VendorID,PU_datetime,DO_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount_neg,extra_neg,mta_tax_neg,tip_amount_neg,tolls_amount_neg,improvement_surcharge_neg,total_amount_neg,congestion_surcharge_neg,Airport_fee_neg,cbd_congestion_fee_neg,PU_Borough_neg,PU_Zone_neg,PU_service_zone_neg,DO_Borough_neg,DO_Zone_neg,DO_service_zone_neg,vendor_name_neg,payment_method_neg,Rate_neg,duration_mins,fare_amount_pos,extra_pos,mta_tax_pos,tip_amount_pos,tolls_amount_pos,improvement_surcharge_pos,total_amount_pos,congestion_surcharge_pos,Airport_fee_pos,cbd_congestion_fee_pos,PU_Borough_pos,PU_Zone_pos,PU_service_zone_pos,DO_Borough_pos,DO_Zone_pos,DO_service_zone_pos,vendor_name_pos,payment_method_pos,Rate_pos
0,2,2025-05-01 00:08:46,2025-05-01 00:18:08,1.0,2.18,1.0,230,239,4,-12.1,-1.0,-0.5,0.0,0.0,-1.0,-17.85,-2.5,0.0,-0.75,Manhattan,Times Sq/Theatre District,Yellow Zone,Manhattan,Upper West Side South,Yellow Zone,"Curb Mobility, LLC",Dispute,Standard,9.366667,12.1,1.0,0.5,0.0,0.0,1.0,17.85,2.5,0.0,0.75,Manhattan,Times Sq/Theatre District,Yellow Zone,Manhattan,Upper West Side South,Yellow Zone,"Curb Mobility, LLC",Dispute,Standard
1,2,2025-05-01 00:18:09,2025-05-01 00:28:49,1.0,1.29,1.0,161,50,4,-11.4,-1.0,-0.5,0.0,0.0,-1.0,-17.15,-2.5,0.0,-0.75,Manhattan,Midtown Center,Yellow Zone,Manhattan,Clinton West,Yellow Zone,"Curb Mobility, LLC",Dispute,Standard,10.666667,11.4,1.0,0.5,0.0,0.0,1.0,17.15,2.5,0.0,0.75,Manhattan,Midtown Center,Yellow Zone,Manhattan,Clinton West,Yellow Zone,"Curb Mobility, LLC",Dispute,Standard
2,2,2025-05-01 00:11:09,2025-05-01 00:15:04,1.0,0.59,1.0,100,162,4,-5.8,-1.0,-0.5,0.0,0.0,-1.0,-11.55,-2.5,0.0,-0.75,Manhattan,Garment District,Yellow Zone,Manhattan,Midtown East,Yellow Zone,"Curb Mobility, LLC",Dispute,Standard,3.916667,5.8,1.0,0.5,0.0,0.0,1.0,11.55,2.5,0.0,0.75,Manhattan,Garment District,Yellow Zone,Manhattan,Midtown East,Yellow Zone,"Curb Mobility, LLC",Dispute,Standard


In [ ]:
merged_cleaned = remove_cancelled_fare_pairs(merged, overall_canceled_rides) # filtering out cancelled rides from the merged df

In [ ]:
merged_cleaned['fare_amount'].mode()  # most common fare amount, $8.6

0    8.6
Name: fare_amount, dtype: float64

In [ ]:
merged_cleaned['fare_amount'].mean()  # overall average fare amount, $20.30

20.3021252392699

> Will be using the average & quantile as a baseline for what's categorized as being cheaper or pricier

In [80]:
# aggregating fare amounts by boroughs
merged_cleaned.groupby(['PU_Borough', 'DO_Borough']).agg({'fare_amount': ['mean', 'median']}).reset_index().sort_values(by=('fare_amount', 'mean'), ascending=False).head()

PU_Borough     DO_Borough fare_amount       
                                        mean median
29  Staten Island        Unknown  260.600000  260.6
23         Queens        Unknown  114.818490   98.2
34        Unknown  Staten Island  112.340000  101.0
22         Queens  Staten Island  102.901079   99.6
4           Bronx  Staten Island   93.900000   99.3

> The mean and median and relatively similar, showing that the data isn't skewed, we can continue with looking at how each borough compares to the average pricing as a whole

> Additionally, the fare prices seem to be mainly based off of geographical distance, which makes sense
* There may not be much to gather of whether there are trends in pricing based off different PU and DO boroughs (inter-borough), which is why we'll look into results strictly within the specific borough

In [ ]:
# categorizing the fare amounts into bins to decide which boroughs are above or below average
# using the overall mean and median fare amounts to define the bins
overall_avg_fare = merged_cleaned['fare_amount'].mean()

# average fare amount for rides within the same borough
borough_fares = (
    merged_cleaned[merged_cleaned['PU_Borough'] == merged_cleaned['DO_Borough']]
    .groupby(['PU_Borough', 'DO_Borough'])['fare_amount']
    .mean()
    .reset_index(name='average_fare')
    )
borough_fares['overall_avg_fare'] = overall_avg_fare  # adding overall average fare column for comparison

bins = [0, overall_avg_fare, merged_cleaned['fare_amount'].max()]
labels = ['Below Average', 'Above Average']

borough_fares['fare_category'] = pd.cut(borough_fares['average_fare'], bins=bins, labels=labels, right=True)
borough_fares[['PU_Borough', 'DO_Borough', 'average_fare', 'overall_avg_fare','fare_category']].drop_duplicates().sort_values(by='average_fare', ascending=False).head(10)

,PU_Borough,DO_Borough,average_fare,overall_avg_fare,fare_category
5,Unknown,Unknown,48.257045,20.302125,Above Average
3,Queens,Queens,30.680329,20.302125,Above Average
4,Staten Island,Staten Island,29.237826,20.302125,Above Average
1,Brooklyn,Brooklyn,20.360099,20.302125,Above Average
0,Bronx,Bronx,18.785821,20.302125,Below Average
2,Manhattan,Manhattan,14.900225,20.302125,Below Average


> It seems like the above average fares take place in Queens, and Staten Island. Because we are focusing on only the 5 boroughs, EWR will be disregarded

> Brooklyn & Bronx seem to be closer to the average, while Manhattan is on the cheaper side

* This backs up the geographical data of Queens and Staten Island being more spaced out 
* Now, lets look even closer into the specific zones within the boroughs to look at the varying fares

*Note: I'll be using these functions in the data_utils pyscript*
* **'categorize_zones':**
  * categorize zones based on average fare amounts and their quartiles returning 3 dataframes: 
    * **pricey_zones**: zones with fares above the 75th percentile of borough average fare
    * **cheap_zones**: zones with fares below the 25th percentile of borough average fare
    * **avg_zones**: zones with fares between the 25th and 75th percentiles of borough average fare 

In [ ]:
# starting with Manhattan, since it has the most rides
# first, filtering out the canceled/refunded rides to get accurate aggregations 
m_canceled_rides = identify_cancelled_rides(manhattan)
manhattan_cleaned = remove_cancelled_fare_pairs(manhattan, m_canceled_rides)

In [ ]:
m_avg_fare = manhattan_cleaned['fare_amount'].mean() # average fare amount in Manhattan
m_pu = manhattan_cleaned.groupby('PU_Zone')['fare_amount'].mean().reset_index(name='avg_fare') # average fare amount by PU
m_do = manhattan_cleaned.groupby('DO_Zone')['fare_amount'].mean().reset_index(name='avg_fare') # average fare amount by DO

In [ ]:
# using the categorize_zones function to categorize the zones based on average fare amounts and their quartiles
m_pricier_zones = categorize_zones(m_pu, m_do, m_avg_fare)[0]
m_cheap_zones = categorize_zones(m_pu, m_do, m_avg_fare)[1]
m_avg_zones = categorize_zones(m_pu, m_do, m_avg_fare)[2]
type(m_pricier_zones), type(m_cheap_zones), type(m_avg_zones)  # checking to make sure the output is a list of DataFrames

In [ ]:
m_pricier_zones.head() # lets look into the pricier zones first

,pricier_zones
0,Battery Park
1,Financial District South
2,Highbridge Park
3,Inwood
4,Inwood Hill Park


In [58]:
# looking into fares where the zones are pricier
m_expensive_neighborhoods = manhattan_cleaned[manhattan_cleaned['PU_Zone'].isin(m_pricier_zones['pricier_zones']) | manhattan_cleaned['DO_Zone'].isin(m_pricier_zones['pricier_zones'])]
m_expensive_neighborhoods['fare_amount'].describe() # checking the variability of fare amounts in pricier zones

count    95626.000000
mean        25.134873
std         11.933424
min          0.000000
25%         17.000000
50%         24.700000
75%         32.500000
max        280.000000
Name: fare_amount, dtype: float64

In [59]:
m_expensive_neighborhoods['fare_amount'].value_counts().sort_values(ascending=False).head(10) # most common fare amounts in expensive zones

fare_amount
27.5    1219
24.0    1123
26.1    1121
26.8    1121
24.7    1088
28.2    1083
29.6    1070
28.9    1070
30.3    1053
25.4    1051
Name: count, dtype: int64

> Even at the 25th percentile, it's $17. On average, the fare is around $25, the mode shows that most fares are in the mid to high $20 

In [60]:
# looking into cheaper zones
m_cheap_zones.head()

,cheaper_zones
0,Flatiron
1,Lenox Hill West
2,Lincoln Square East
3,Upper East Side North
4,Upper East Side South


In [61]:
# looking into fares 
m_cheap_neighborhoods = manhattan_cleaned[manhattan_cleaned['PU_Zone'].isin(m_cheap_zones['cheaper_zones']) | manhattan_cleaned['DO_Zone'].isin(m_cheap_zones['cheaper_zones'])]
m_cheap_neighborhoods['fare_amount'].describe() # checking the variability of fare amounts in cheaper zones

count    1.562677e+06
mean     1.402071e+01
std      7.785098e+00
min      0.000000e+00
25%      8.600000e+00
50%      1.210000e+01
75%      1.770000e+01
max      9.000000e+02
Name: fare_amount, dtype: float64

In [62]:
m_cheap_neighborhoods['fare_amount'].value_counts().sort_values(ascending=False).head(10)  

fare_amount
7.9     75563
8.6     75548
9.3     74409
10.0    73253
7.2     72343
10.7    68959
11.4    65970
6.5     65465
12.1    61821
12.8    57334
Name: count, dtype: int64

In [63]:
# percentage of fares below $14.02 (the average in cheaper zones)
(m_cheap_neighborhoods[m_cheap_neighborhoods['fare_amount'] < 14.02]['fare_amount'].count()/ m_cheap_neighborhoods.shape[0]) * 100  

59.58972967542236

> On average the fare is around $14, with almost 60% of the fares under that amount

In [64]:
# looking at the average zones
m_avg_zones.head()

,average_zones
0,Alphabet City
1,Battery Park City
2,Bloomingdale
3,Central Harlem
4,Central Harlem North


In [65]:
# looking into fares 
m_avg_neighborhoods = manhattan_cleaned[manhattan_cleaned['PU_Zone'].isin(m_avg_zones['average_zones']) | manhattan_cleaned['DO_Zone'].isin(m_avg_zones['average_zones'])]
m_avg_neighborhoods['fare_amount'].describe() # checking the variability of fare amounts in average zones

count    3.240653e+06
mean     1.511212e+01
std      8.612107e+00
min      0.000000e+00
25%      9.300000e+00
50%      1.350000e+01
75%      1.910000e+01
max      7.000000e+02
Name: fare_amount, dtype: float64

In [66]:
m_avg_neighborhoods['fare_amount'].value_counts().sort_values(ascending=False).head(10) 

fare_amount
9.3     135246
10.0    135183
8.6     134040
10.7    129600
7.9     128679
11.4    125288
7.2     119421
12.1    119003
12.8    112599
13.5    104605
Name: count, dtype: int64

In [67]:
# percentage of fares below $14.02 (the average in cheaper zones)
(m_avg_neighborhoods[m_avg_neighborhoods['fare_amount'] < 15.11]['fare_amount'].count()/ m_avg_neighborhoods.shape[0]) * 100  

60.36561149867018

> the average fare for the average zone is $15, with 60% less than that amount.
* within the average zones, the fares seem to lean on the lesser side 

In [ ]:
# turn all the steps above into a function to make it reusable for other boroughs
# queens

> Based on the average fares & percentiles, those were the different categories of whether the neighborhood is: pricier, average, or cheaper

> Lets investigate tipping behaviors according to each of the categories to see if there are any trends:

> Generally, which areas have the highest/lowest average tips?
* Using categorized zones to analyze tipping behavior in Manhattan

In [69]:
manhattan_cleaned['tip_amount'].agg(['mean', 'min', 'max'])  # average tips

mean      2.411957
min       0.000000
max     443.210000
Name: tip_amount, dtype: float64

In [70]:
manhattan_cleaned['tip_amount'].value_counts().sort_values(ascending=False).head(10)  # most common tip amounts

tip_amount
0.00    1098658
2.00     171453
1.00     131457
3.00      71143
1.50      32300
4.00      30667
5.00      30207
2.95      26732
3.15      24888
3.65      21877
Name: count, dtype: int64

In [71]:
manhattan_cleaned[manhattan_cleaned['tip_amount'] == 0].shape[0] / manhattan_cleaned.shape[0] * 100  # percentage of rides with no tips

32.491783585269864

> the average tip is $2.41, around 32.5% of tip are 0

In [72]:
# highest average tips
m_pu_avg_tips = manhattan_cleaned.groupby('PU_Zone')['tip_amount'].mean().reset_index(name='avg_tip').sort_values(by='avg_tip', ascending=False)
m_do_avg_tips = manhattan_cleaned.groupby('DO_Zone')['tip_amount'].mean().reset_index(name='avg_tip').sort_values(by='avg_tip', ascending=False)
m_high_tip_zones = pd.merge(m_pu_avg_tips, m_do_avg_tips, left_on='PU_Zone', right_on='DO_Zone', suffixes=('_PU', '_DO'))
m_high_tip_zones[['PU_Zone', 'avg_tip_PU', 'avg_tip_DO']].head()

,PU_Zone,avg_tip_PU,avg_tip_DO
0,Randalls Island,7.895607,4.574159
1,Battery Park,3.034561,2.778146
2,Battery Park City,3.018217,3.072819
3,World Trade Center,2.918473,2.710069
4,Penn Station/Madison Sq West,2.899933,2.317618


In [73]:
# lowest average tips
m_high_tip_zones[['PU_Zone', 'avg_tip_PU', 'avg_tip_DO']].tail()


,PU_Zone,avg_tip_PU,avg_tip_DO
61,Washington Heights North,0.418266,2.283107
62,Central Harlem North,0.339869,1.523421
63,Inwood,0.196783,1.805343
64,Marble Hill,0.183167,1.903147
65,Highbridge Park,0.149744,0.963459


> Whether tipping behaviors seem constant within their respective categories:
* Do expensive neighbourhoods have higher tips compared to the rest?
* Which areas have the most consistent tipping behaviors

In [74]:
# finding what percentage of rides in their respective categories are above the general average tip amount in Manhattan
m_avg_tip = manhattan_cleaned['tip_amount'].mean()
(m_expensive_neighborhoods['tip_amount'] >= m_avg_tip).sum() / m_expensive_neighborhoods.shape[0] * 100  # percentage of rides with tips above the average tip amount in expensive zones

30.16334469704892

In [75]:
m_avg_tip = manhattan_cleaned['tip_amount'].mean()
(m_avg_neighborhoods['tip_amount'] >= m_avg_tip).sum() / m_avg_neighborhoods.shape[0] * 100  # percentage of rides with tips above the average tip amount in expensive zones

49.79576647052307

In [76]:
m_avg_tip = manhattan_cleaned['tip_amount'].mean()
(m_cheap_neighborhoods['tip_amount'] >= m_avg_tip).sum() / m_cheap_neighborhoods.shape[0] * 100  # percentage of rides with tips above the average tip amount in cheap zones

51.52638709087035

In [77]:
m_expensive_neighborhoods['tip_amount'].describe(), m_avg_neighborhoods['tip_amount'].describe(), m_cheap_neighborhoods['tip_amount'].describe()  

(count    95626.000000
 mean         2.009848
 std          3.340597
 min          0.000000
 25%          0.000000
 50%          0.000000
 75%          3.950000
 max         98.000000
 Name: tip_amount, dtype: float64,
 count    3.240653e+06
 mean     2.426328e+00
 std      2.412487e+00
 min      0.000000e+00
 25%      0.000000e+00
 50%      2.390000e+00
 75%      3.850000e+00
 max      4.432100e+02
 Name: tip_amount, dtype: float64,
 count    1.562677e+06
 mean     2.494381e+00
 std      2.211765e+00
 min      0.000000e+00
 25%      0.000000e+00
 50%      2.500000e+00
 75%      3.730000e+00
 max      2.220000e+02
 Name: tip_amount, dtype: float64)

> From least variability (most consisten) to the most: cheap, average, expensive neighborhoods

> For Manhattan:
> It seems that amount being tipped isn't relative/constant to its categorized zone
* In fact, it had the opposite effect
  * Instead of more expensive neighborhoods having tipped more, the cheaper and average neighborhoods tipped more 
  * The standard deviation shows that the cheaper and average neighborhoods' tipping was most consistent 
  * Cheaper fares --> more tip, Pricier fares --> less tip
  

In [ ]:
# turn into a function to make it reusable for other boroughs
# queens

> Consider looking at airports? Do they get tipped more on average?

#### **Key Findings:**

---
#### **2. Time Segmentation & Tipping Behavior**
* Let's take a closer look at whether there are datetime trends:
  * Weekends (people could be going out more)
  * Maybe there are activities after working hours (events, personal outings)
  * Time of day: morning, afternoon, night

In [79]:
# lets create bin to categorize the datetime data